In [4]:
import numpy as np
import pandas as pd
import math
import scipy as sc
from scipy.stats import norm
from scipy.special import roots_hermite
# import keras
# from keras.models import Sequential
# from keras.layers import Dense
# import keras.optimizers as opt
# from keras.constraints import Constraint
import sobol_seq
import time
# import tensorflow as tf
# from keras import backend as keras_backend
import sys
import matplotlib.pyplot as plt
import seaborn as sns
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly
plotly.offline.init_notebook_mode(connected=True)
from itertools import chain, product 


num_cores = 32
#No.of.derivative contracts used for static hedging
no_of_options=15

#No of paths
no_of_paths = 50000
no_of_paths_test = 50000

# Total number of runs neural network static hedge created
no_of_runs = 4

# num_CPU = 1
# num_GPU = 0

# config = tf.ConfigProto(intra_op_parallelism_threads=num_cores, inter_op_parallelism_threads=num_cores,
#                         allow_soft_placement=True, device_count={'CPU': num_CPU, 'GPU': num_GPU})
# session = tf.Session(config=config)
# keras_backend.set_session(session)

##################################################################################
# Stability Analysis of NN Static Hedging and comparison with Carr's Static Hedge#
##################################################################################

#Current Stock Price
S0=100

#Risk Free Interest Rate
r=0.06 

#Continuos Divident rate
delta=0.02

#Volatility
sigma=0.27 

# Time to maturity for the shorter term options = 0.25 yrs 
t=0.25  

# Time to maturity for the target options = 1 yr
T=1

# Size of each step = 0.01
dt=0.01

# The call was struck at the money on 11/08/2020
K=100 


# No. of sim time points
sim_grid_points = int(t/dt)

#No. of. time points
sim_points = [dt * i for i in range(1, sim_grid_points + 1)]



In [5]:
#Loading Datasets

#Stock Simulation - Testing
df_sim_stocks_test = pd.read_csv('Stability Analysis - Outputs/Dataframes Optimized/Stock Simulations/df_sim_stocks_test_sim' + str(no_of_paths_test) + '.csv', index_col=0)

#BS Price - for Testing Sample
df_bs_price_test = pd.read_csv('Stability Analysis - Outputs/Dataframes Optimized/df_bs_price_test_sim' + str(no_of_paths_test) + '.csv', index_col=0)

#Carr Static Hedge across simulations
df_carr_pv_rmse = pd.read_csv('Stability Analysis - Outputs/Dataframes Optimized/df_carr_pv_rmse_sim' + str(no_of_paths) + '.csv', index_col=0)
df_carr_hedge_error = pd.read_csv('Stability Analysis - Outputs/Dataframes Optimized/df_carr_hedge_error_sim' + str(no_of_paths) + '.csv', index_col=0)

#Loading Neural Network Static Hedge
file_ident = "constr_normal"
# file_ident = "unconstr"

df_nn_hedge_pv_instr_nos = pd.read_csv('Stability Analysis - Outputs/Dataframes Optimized/df_nn_hedge_pv_instr_nos_' + file_ident + '_sim' + str(no_of_paths) + '_opt' + str(no_of_options) + '_runs' + str(no_of_runs) + '.csv', index_col=0)
df_nn_hedge_params = pd.read_csv('Stability Analysis - Outputs/Dataframes Optimized/df_nn_hedge_params_' + file_ident + '_sim' + str(no_of_paths) + '_opt' + str(no_of_options) + '_runs' + str(no_of_runs) + '.csv', index_col=0)
df_nn_hedge_error = pd.read_csv('Stability Analysis - Outputs/Dataframes Optimized/df_nn_hedge_error_' + file_ident + '_sim' + str(no_of_paths) + '_opt' + str(no_of_options) + '_runs' + str(no_of_runs) + '.csv', index_col=0)



In [6]:
# Function for Four Plots 

def generate_plot_inputs(df_sim_stocks_test, df_carr_pv_rmse, df_carr_hedge_error, 
                         df_nn_hedge_pv_instr_nos, df_nn_hedge_error, 
                         no_of_options, no_of_paths_test, no_of_plots=4):
    
    #Error Vector and MSE
    stock_vector_list = [np.array(df_sim_stocks_test.iloc[-1, :]).reshape(-1) for i in range(0, no_of_plots)]
    carr_static_hedge_error = [np.absolute(np.array(df_carr_hedge_error.loc[:,'Options Count:' + str(no_of_options)]).reshape(-1))  for i in range(0, no_of_plots)]
    nn_static_hedge_error = [np.absolute(np.array(df_nn_hedge_error.iloc[:,i]).reshape(-1)) for i in range(0, no_of_plots)]
    carr_static_hedge_rmse = [np.array(df_carr_pv_rmse[df_carr_pv_rmse['Options Count'] == no_of_options]['Carr RMSE']) for i in range(0, no_of_plots)]
    nn_static_hedge_rmse = [np.sqrt(np.mean(np.square(error_array))) for error_array in nn_static_hedge_error]
    x_list = stock_vector_list
    y_list = [[carr_static_hedge_error[i], nn_static_hedge_error[i]] for i in range(0, no_of_plots)]
    
    #Titles
    figure_title_str = "Static Hedge Error across Stock Levels Analysis"
    
    #Sub title names
    subplot_title_str_list = ["Independent Neural Network Run: " + str(i) for i in range(1, no_of_plots+1)]
    
    #Legends
    legends_str = [["Carr Static Hedge", "RLNN Static Hedge"] for i in range(1, no_of_plots+1)]
    
    
    #Axes Titles 
    xaxis_title_str = ["Index/Stock price" for i in range(0, no_of_plots)]
    yaxis_title_str = ["Hedging Error" for i in range(0, no_of_plots)]
    
    #Correct x-axis and y axis title to avoid them in between plots - if all plots on same axis:
    for i in range(0, no_of_plots):
        if (i==0):
            xaxis_title_str[i] = ""
        elif (i==1):
            xaxis_title_str[i] = ""
            yaxis_title_str[i] = ""
        elif(i==3):
            yaxis_title_str[i] = ""
        else:
            None
            
    #Limits
    x_limits = [[np.amin(stock_vector_list[i]), np.amax(stock_vector_list[i])] for i in range(0, no_of_plots)]
    carr_limits = [[np.amin(carr_static_hedge_error[i]), np.amax(carr_static_hedge_error[i])] for i in range(0, no_of_plots)]
    nn_limits = [[np.amin(nn_static_hedge_error[i]), np.amax(nn_static_hedge_error[i])] for i in range(0, no_of_plots)]
    y_limits = [[np.amin([carr_limits[i][0], nn_limits[i][0]]) - 1, np.amax([carr_limits[i][1], nn_limits[i][1]]) + 1]  for i in range(0, no_of_plots)]
    
    #Other annotations
    other_annotations = ["Carr RMSE: " + str(round(carr_static_hedge_rmse[i][0], 2)) + "<br>" "NN RMSE: " + str(round(nn_static_hedge_rmse[i], 2)) for i in range(0, no_of_plots)]
    
    return figure_title_str, subplot_title_str_list, xaxis_title_str, yaxis_title_str, legends_str, other_annotations, x_limits, y_limits, x_list, y_list 
    

#Inputs:

#     1. no_of_plots: int --> No. of. plots in the figure
#     2. no_of_rows: int --> No. of. rows in the figure
#     3. no_of_cols: int --> No. of. columns in the figure
#     4. no_of_yvars: int --> No. of. functions in each sub plots
#     5. figure_title_str: str --> Figure Title
#     6. subplot_title_str_list: list of strings with length no_of_plots --> Each string for each subplot's title
#     7. xaxis_title_str: list:  x-axis title string for each plot
#.    8. yaxis_title_str: list:  y-axis title string for each plot
#.    9. legends_str: list --> [[y1_legend, y2_legend,... ], ..] --> list of list of y_axes legends for each y-axis variable and each plot
#     7. x_list: list --> List of x axis numpy arrays for each sub-plot
#     8. y_list: list ---> contains no_of_plots times list. Each list has numpy arrays each y-axis plot i.e. multiple functions in one plot
    
def generate_4plots(no_of_plots, no_of_rows, no_of_cols, no_of_yvars, 
                    figure_title_str, subplot_title_str_list, xaxis_title_str, yaxis_title_str, legends_str,
                    other_annotations, x_limits, y_limits, 
                    x_list, y_list):

    fig = make_subplots(rows=no_of_rows, cols=no_of_cols, subplot_titles=subplot_title_str_list)
    fig.update_layout(height=700, width=1000,
                      title=dict(text = figure_title_str, y=0.95, x=0.42, xanchor='center', yanchor='top'), font=dict(color="black", size=18), 
                      showlegend=True)
    
    figure_positions = list(product(range(1, no_of_rows + 1), range(1, no_of_cols + 1)))
    colours_list = ['rgb(139, 0, 0)', 'rgb(0, 0, 139)', 'rgb(0, 139, 0)']
    legend_show_list = [True] + [False for i in range(0, no_of_plots-1)]
    
    for i in range(0, no_of_plots):
        for j in range(0, no_of_yvars):
            fig.add_trace(go.Scatter(x=list(x_list[i]), y=list(y_list[i][j]), 
                          mode = 'markers', marker=dict(size=4, color = colours_list[j]), name=legends_str[i][j], showlegend=legend_show_list[i]),
                          row=figure_positions[i][0], col=figure_positions[i][1])
    
    for i in range(0, no_of_plots):
        fig.update_xaxes(title_text=xaxis_title_str[i], showgrid=True, range=x_limits[i], row=figure_positions[i][0], col=figure_positions[i][1])
        fig.update_yaxes(title_text=yaxis_title_str[i], showgrid=True, range=y_limits[i], row=figure_positions[i][0], col=figure_positions[i][1])

    annotations = []
    annotations_extend=[dict(x=0.01, y=0.999, xref="paper", yref="paper", 
                  text=other_annotations[0], 
                  showarrow=False, xanchor="left", yanchor="top",  font=dict(color="black", size=8)),
             dict(x=0.56, y=0.999, xref="paper", yref="paper", 
                  text=other_annotations[1], 
                  showarrow=False, xanchor="left", yanchor="top", font=dict(color="black", size=8)), 
             dict(x=0.01, y=0.37, xref="paper", yref="paper", 
                  text=other_annotations[2], 
                  showarrow=False, xanchor="left", yanchor="top", font=dict(color="black", size=8)),
             dict(x=0.56, y=0.37, xref="paper", yref="paper", 
                  text=other_annotations[3], 
                  showarrow=False, xanchor="left", yanchor="top", font=dict(color="black", size=8)), 
                    ]
    annotations = [annotation for annotation in fig["layout"]["annotations"]]
    annotations = annotations + annotations_extend
    fig["layout"]["annotations"] += tuple(annotations)
    
    return fig
    
figure_title_str, subplot_title_str_list, xaxis_title_str, yaxis_title_str, legends_str, other_annotations, x_limits, y_limits, x_list, y_list = generate_plot_inputs(df_sim_stocks_test, df_carr_pv_rmse, df_carr_hedge_error, df_nn_hedge_pv_instr_nos, df_nn_hedge_error, no_of_options, no_of_paths_test, no_of_plots=4) 
final_plot = generate_4plots(no_of_plots=4, no_of_rows=2, no_of_cols=2, no_of_yvars=2, 
                             figure_title_str=figure_title_str, subplot_title_str_list=subplot_title_str_list, xaxis_title_str=xaxis_title_str, yaxis_title_str=yaxis_title_str, legends_str=legends_str,
                             other_annotations=other_annotations, x_limits=x_limits, y_limits=y_limits, 
                             x_list=x_list, y_list=y_list) 
# final_plot.show()

plot_name = 'Final Outputs - Static Hedge Comments/Stability of Neural Network/Static Hedge Comparisons/fig_nn_hedge_error_' + file_ident + '_sim' + str(no_of_paths) + '_opt' + str(no_of_options) + '_runs' + str(no_of_runs) + '.png'
final_plot.write_image(plot_name)


In [5]:
import plotly.graph_objects as go
import pandas as pd
import plotly.express as px


no_of_options=5
no_of_paths = 5000
no_of_paths_test = 5000

# Total number of runs neural network static hedge created
no_of_runs = 100

file_ident = "constr_normal"
# file_ident = "constr_nomal_fixSeed"
df = pd.read_csv('Stability Analysis - Outputs/Dataframes Optimized/df_nn_hedge_params_' + file_ident + '_sim' + str(no_of_paths_test) + '_opt' + str(no_of_options) + '_runs' + str(no_of_runs) + '.csv', index_col=0)
df['rank'] = df.groupby('Runs')['Strike'].rank(method='max')

fig = go.Figure()

for i in range(1, no_of_options+1, 1):
    fig.add_trace(go.Histogram(x=df[df['rank']==i]['Strike'], name= "Strike " + str(int(i))))
fig.update_layout(barmode='overlay')

# Reduce opacity to see both histograms
fig.update_traces(opacity=0.6)

fig.update_layout(
    title_text='Distribution of Call option Strikes (100 runs) of Neural Network - ' + str(no_of_options) + ' options', # title of plot
    title_x = 0.48,
    title_y = 0.89,
    xaxis_title_text='Strike', # xaxis label
    yaxis_title_text='Count', # yaxis label
    bargap=0.05, # gap between bars of adjacent location coordinates
    bargroupgap=0 # gap between bars of the same location coordinates
)



fig.show()

# file_ident = "nn_constr_call"
# file_ident = "constr_nomal_fixSeed"
# plot_name = 'fig_' + file_ident + '_sim' + str(no_of_paths_test) + '_opt' + str(no_of_options) + '_runs' + str(no_of_runs)

# fig.write_image("Stability Analysis - Outputs/Final Plots/" + plot_name + ".png")


In [32]:

no_of_options=5
no_of_paths = 5000
no_of_paths_test = 5000

# Total number of runs neural network static hedge created
no_of_runs = 100

file_ident1 = "constr_normal"
file_ident2 = "constr_nomal_fixSeed"
df = pd.read_csv('Stability Analysis - Outputs/Dataframes Optimized/df_nn_hedge_params_' + file_ident1 + '_sim' + str(no_of_paths_test) + '_opt' + str(no_of_options) + '_runs' + str(no_of_runs) + '.csv', index_col=0)
df['rank'] = df.groupby('Runs')['Strike'].rank(method='max')
df2 = pd.read_csv('Stability Analysis - Outputs/Dataframes Optimized/df_nn_hedge_params_' + file_ident2 + '_sim' + str(no_of_paths_test) + '_opt' + str(no_of_options) + '_runs' + str(no_of_runs) + '.csv', index_col=0)
df2['rank'] = df2.groupby('Runs')['Strike'].rank(method='max')
strikes_str_list = ["Strike " + str(i) for i in range(1, no_of_options + 1)]
# x_axis_range = [[int(np.amin(np.array(df[df['rank']==i]['Strike']))), int(np.amax(np.array(df[df['rank']==i]['Strike'])))] for i in range(1,no_of_options+1)]
x_axis_range = [[68,69], [83,86], [93,96], [104,107], [119,121]]
fig = make_subplots(rows=5, cols=2, subplot_titles=["Strikes - Random Seed", "Strikes - Fixed Seed", "", "", "", "", "", "", "", ""])
x_bins_list = [dict(start=68,end=69,size=0.2), dict(start=83,end=86,size=0.25), 
               dict(start=93,end=96,size=0.25), dict(start=104,end=107,size=0.25), dict(start=119,end=121,size=0.25)]

Mean_list_random_seed = [round(np.mean(np.array(df[df['rank']==i]['Strike'])),2) for i in range(1, no_of_options+1)]
Mean_list_fixed_seed = [round(np.mean(np.array(df2[df2['rank']==i]['Strike'])),2) for i in range(1, no_of_options+1)]
StdDev_list_random_seed = [round(np.std(np.array(df[df['rank']==i]['Strike'])),2) for i in range(1, no_of_options+1)]
StdDev_list_fixed_seed = [round(np.std(np.array(df2[df2['rank']==i]['Strike'])),2) for i in range(1, no_of_options+1)]

fig.update_layout(height=1000, width=1000,
                title=dict(text = "Histogram of Neural Network Normal Initiated Strikes", y=0.96, x=0.5, xanchor='center', yanchor='top'), font=dict(color="black", size=18), 
                      showlegend=False)
    
for i in range(1, no_of_options+1, 1):
    fig.add_trace(go.Histogram(x=df[df['rank']==i]['Strike'], name= "Strike " + str(int(i)), 
                              xbins=x_bins_list[i-1]), row=i, col=1)

for i in range(1, no_of_options+1, 1):
    fig.add_trace(go.Histogram(x=df2[df2['rank']==i]['Strike'], name= "Strike " + str(int(i)), 
                              xbins=x_bins_list[i-1]), row=i, col=2)


for i in range(1, no_of_options+1):
        fig.update_yaxes(title_text=strikes_str_list[i-1], showgrid=True, row=i, col=1)

fig.update_xaxes(tickangle=30, showgrid=False)
fig.update_yaxes(showgrid=False)

for i in range(1, no_of_options+1):
    fig.update_xaxes(range=x_axis_range[i-1], row=i, col=1)
    fig.update_xaxes(range=x_axis_range[i-1], row=i, col=2)

fig.update_layout(barmode='overlay')
fig.update_layout(
    bargap=0.05, # gap between bars of adjacent location coordinates
    bargroupgap=0 # gap between bars of the same location coordinates
)

fig.layout.annotations[0].update(x=0.25, y=1, font={'size':19})
fig.layout.annotations[1].update(x=0.75, y=1, font={'size':19})


y_location = [0.999, 0.78, 0.56, 0.34, 0.12]

annotations = []
annotations_extend=[dict(x=0.01, y=y_location[i], xref="paper", yref="paper", 
                  text="Mean: " + str(Mean_list_random_seed[i]) + "<br> Std Dev: " + str(StdDev_list_random_seed[i]), 
                  showarrow=False, xanchor="left", yanchor="top",  font=dict(color="black", size=12)) 
                   for i in range(0, no_of_options)] + [dict(x=0.56, y=y_location[i], xref="paper", yref="paper", 
                  text="Mean: " + str(Mean_list_fixed_seed[i]) + "<br> Std Dev: " + str(StdDev_list_fixed_seed[i]), 
                  showarrow=False, xanchor="left", yanchor="top",  font=dict(color="black", size=12)) 
                   for i in range(0, no_of_options)]
                
annotations = [annotation for annotation in fig["layout"]["annotations"]]
annotations = annotations + annotations_extend
fig["layout"]["annotations"] += tuple(annotations)

fig.show()

file_ident = "hist_strikes_same_axes"
plot_name = 'fig_' + file_ident + '_sim' + str(no_of_paths_test) + '_opt' + str(no_of_options) + '_runs' + str(no_of_runs)
fig.write_image("Stability Analysis - Outputs/Final Plots/" + plot_name + ".png")



In [54]:
[1.174481176421648, 0.9238793099104897, 1.0319039096057099, 0.975062962452259, 1.1462107937044974]

[1.174481176421648,
 0.9238793099104897,
 1.0319039096057099,
 0.975062962452259,
 1.1462107937044974]